In [696]:
#Importing Libraries
import pandas as pd 
from datetime import datetime
import xlwt 
from xlwt import Workbook 
import numpy as np
import math
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [697]:
#Importing datasets

#Training data
cgm_P1 = pd.read_csv('CGMData.csv')
ins_P1 = pd.read_csv('InsulinData.csv')


/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (32) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (13,43) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [698]:
#Getting tm times:
def get_tm_times(ins_df):
    tm_list_date = []
    tm_list_time = []
    tm_list_row = []

    for i in range(len(ins_df),0,-1):
        if(np.isnan(ins_df.loc[i-1,"BWZ Carb Input (grams)"]) == False): #Searching "Y" column for non NAN values
            if(i-1 < len(ins_df) - 7): #Catching index out of bounds error
                tm_list_date.append(ins_df.loc[i-1+6 ,"Date"]) #appending start date (tm - 30min)
                tm_list_date.append(ins_df.loc[i-1-24,"Date"])#appending end date (2 hours later)

                tm_list_time.append(ins_df.loc[i-1+6,"Time"]) #appending start time (t - 30min)
                tm_list_time.append(ins_df.loc[i-1-24,"Time"]) #appending end time (tm + 2hrs)

                tm_list_row.append(i-1+6) #appending start row (t - 30min)
                tm_list_row.append(i-1-24) #appending end row (tm + 2hrs)
        
    return tm_list_date, tm_list_time, tm_list_row

In [699]:
def indicate_meal_boundaries(tm_list_date, tm_list_time, tm_list_row):
    #3 conditions:
    meal = False
    for i in range(0, len(tm_list_row), 2): 
        for j in range(1, 25): #Checking next 2 hours for a meal:
            if(np.isnan(ins_P1.loc[tm_list_row[i] - j - 6, "BWZ Carb Input (grams)"]) == False):
                meal = True #There is another meal, so a) condition fails
                #Handling condition b)
                tm_list_date[i] = ins_P1.loc[tm_list_row[i] - j, "Date"] #appending new start date of meal
                tm_list_date[i+1] = ins_P1.loc[tm_list_row[i] - j - 30, "Date"] #appending new end date of meal

                tm_list_time[i] = ins_P1.loc[tm_list_row[i] - j, "Time"] #appending new start time of meal
                tm_list_time[i+1] = ins_P1.loc[tm_list_row[i] - j - 30, "Time"] #appending new end time of meal

                tm_list_row[i] = tm_list_row[i] - j #appending new start row of meal
                tm_list_row[i+1] = tm_list_row[i] - 30 #appending new end row of meal
                break
            if(j == 24 and np.isnan(ins_P1.loc[tm_list_row[i] - j - 6, "BWZ Carb Input (grams)"]) == False): 
                #Handling condition c) #18 is tm + 1hr 30min, 48 is tm + hrs
                tm_list_date[i] = ins_P1.loc[tm_list_row[i] - j - 18, "Date"] #appending new start date of meal
                tm_list_date[i+1] = ins_P1.loc[tm_list_row[i] - 48 , "Date"] #appending new end date of meal

                tm_list_time[i] = ins_P1.loc[tm_list_row[i] - j - 18, "Time"] #appending new start time of meal
                tm_list_time[i+1] = ins_P1.loc[tm_list_row[i] - 48, "Time"] #appending new end time of meal

                tm_list_row[i] = tm_list_row[i] - 18 #appending new start row of meal
                tm_list_row[i+1] = tm_list_row[i] - 30 #appending new end row of meal
    return tm_list_date, tm_list_time, tm_list_row

In [700]:
def indicate_no_meal_boundaries(meal_list_date, meal_list_time, meal_list_row, ins_df):
    no_meal_list_date = []
    no_meal_list_time = []
    no_meal_list_row = []
    index = len(ins_df) - 1
    
    j = 0
    while(index >= 0 and j < len(meal_list_row)):
        if(index > meal_list_row[j] + 24): #No meal and meal time can overlap for first 30 min of meal
            if(index - 24 > 0):
                no_meal_list_date.append(ins_df.loc[index, "Date"]) #appending start of no meal date
                no_meal_list_date.append(ins_df.loc[index - 24, "Date"]) #appending end of no meal date
                no_meal_list_time.append(ins_df.loc[index, "Time"]) #appending start of no meal time
                no_meal_list_time.append(ins_df.loc[index - 24, "Time"]) #appending end of no meal time
                no_meal_list_row.append(index) #appending start of no meal row
                no_meal_list_row.append(index - 24) #appending end of no meal row
                index = index - 24
            else:
                break
        else: #Go to end of meal time
            if(meal_list_row[j+1] - 1 - 24 > 0 and j+1 < len(meal_list_row)):
                index = meal_list_row[j+1] - 1 #End of meal time
                no_meal_list_date.append(ins_df.loc[index, "Date"]) #appending start of no meal date
                no_meal_list_date.append(ins_df.loc[index - 24, "Date"]) #appending end of no meal date
                no_meal_list_time.append(ins_df.loc[index, "Time"]) #appending start of no meal time
                no_meal_list_time.append(ins_df.loc[index - 24, "Time"]) #appending end of no meal time
                no_meal_list_row.append(index) #appending start of no meal row
                no_meal_list_row.append(index - 24) #appending end of no meal row
                j = j + 1 #Going to start of next meal
            else:
                break
    return no_meal_list_date, no_meal_list_time, no_meal_list_row

In [701]:
def extract_features(meal_list_date, meal_list_time, meal_list_row, no_meal_list_date, no_meal_list_time, no_meal_list_row, cgm_DF):
    means = [] #Columns: mean, sd, ground truth value
    sds = []
    ground_truths = []
    
    #Collecting cgm data of meals
    for i in range(0, len(meal_list_row) - 1, 2):
        meal_data = []
        j = meal_list_row[i]
        while(j != meal_list_row[i+1] and j >= 0):
            if(math.isnan(cgm_DF.loc[j, "Sensor Glucose (mg/dL)"]) == True):
                pass
                #nan value, so don't append it
            else:
                meal_data.append(cgm_DF.loc[j, "Sensor Glucose (mg/dL)"])
            j = j-1 #decrementing j
            
        #Getting mean and sd of set of data and ground truth
        if(len(meal_data) > 0):
            mean = float(sum(meal_data) / len(meal_data))
            sd = np.std(meal_data, ddof=0)
            means.append(mean)
            sds.append(sd)
            ground_truths.append(1) #1 equals meal and 0 equals no meal
    
    #Collecting cgm value data of no meals
    for i in range(0, len(no_meal_list_row) - 1, 2):
        no_meal_data = []
        j = no_meal_list_row[i]
        while(j != no_meal_list_row[i+1] and j >= 0):
            if(math.isnan(cgm_DF.loc[j, "Sensor Glucose (mg/dL)"]) == True):
                pass
                #nan value, so don't append it
            else:
                no_meal_data.append(cgm_DF.loc[j, "Sensor Glucose (mg/dL)"])
            j = j-1 #decrementing j
            
        #Getting mean and sd of set of data and ground truth
        if(len(no_meal_data) > 0):
            mean = float(sum(no_meal_data) / len(no_meal_data))
            sd = np.std(no_meal_data, ddof=0)
            means.append(mean)
            sds.append(sd)
            ground_truths.append(0) #1 equals meal and 0 equals no meal
    
    return means, sds, ground_truths

In [702]:
def create_features_df(means,sds,ground_truths):
    #creating feature dataframe
    data = {'mean': means,
            'sd': sds,
            'ground_truth': ground_truths
            }
    
    features_df = pd.DataFrame(data, columns = ['mean','sd', 'ground_truth'])
    
    return features_df

In [703]:
def data_preprocessing(features_df):
    x = features_df.iloc[:,0:2].values #Independent variables
    y = features_df.iloc[:,2].values #Dependent variable 
  
    #splitting values into dependent and independent variables
    x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.20, random_state = 0)
    
    #normalizing values
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)

    return x_train, y_train, x_test, y_test

In [704]:
def classification_model(x_train, y_train, x_test, y_test):
    from sklearn.svm import SVC #Using SVM classifier as my machine
    from sklearn.metrics import classification_report
    from numpy import mean
    from numpy import std
    from sklearn.model_selection import KFold
    from sklearn.model_selection import cross_val_score
    
    #Fitting data to model
    svm_model = SVC(kernel='sigmoid', random_state = 1, class_weight = 'balanced', coef0 =0.0)
    svm_model = svm_model.fit(x_train, y_train)
    y_pred = svm_model.predict(x_test)
    
    #Printing out analytics
    #print(classification_report(y_test,y_pred))
    
    #Evaluating model using k-fold cross validation:
    cv = KFold(n_splits=10, random_state=1, shuffle=True)
    scores = cross_val_score(svm_model, x_train, y_train, scoring ='accuracy', cv = cv, n_jobs = -1)
    print('KFold Cross Validation Training data Accuracy: Mean: %.3f Standard Deviation: (%.3f)' % (mean(scores), std(scores)))
    
    return svm_model

In [705]:
def save_model(svm_model):
    import pickle
    
    #Saving model
    assignment_2_model_file = "assignment_2_model.pkl"  
    with open(assignment_2_model_file, 'wb') as file:  
        pickle.dump(svm_model, file)
    
    return

In [706]:
#Driver

#Finding tm times
tm_list_date, tm_list_time, tm_list_row = get_tm_times(ins_P1)

#Finding meal list start and end times
meal_list_date, meal_list_time, meal_list_row = indicate_meal_boundaries(tm_list_date, tm_list_time, tm_list_row)

#Finding no meal list start and end times
no_meal_list_date, no_meal_list_time, no_meal_list_row = indicate_no_meal_boundaries(meal_list_date, meal_list_time, meal_list_row, ins_P1)

#Extracting features from meal and no meal data
means, sds, ground_truths = extract_features(meal_list_date, meal_list_time, meal_list_row, no_meal_list_date, no_meal_list_time, no_meal_list_row, cgm_P1)

#Creating features dataframe as input for ml classification model
features_df = create_features_df(means, sds, ground_truths)

#Data preprocessing 
x_train, y_train, x_test, y_test = data_preprocessing(features_df)

#Creating classification model
svm_model = classification_model(x_train, y_train, x_test, y_test)

#Saving model using Pickle Package
save_model(svm_model)

KFold Cross Validation Training data Accuracy: Mean: 0.506 Standard Deviation: (0.033)


In [707]:
print("Finished Compiling")

Finished Compiling
